In [2]:


import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Dataset
df_fake = pd.read_csv("Fake.csv")  # Load locally
df_real = pd.read_csv("True.csv")  # Load locally


df_fake['target'] = 1  # Fake News

df_real['target'] = 0  # Real News

df = pd.concat([df_fake, df_real])
df = df[['title', 'text', 'target']]
df['text'] = df['title'] + " " + df['text']  # Combine title and text

df.drop(columns=['title'], inplace=True)

df = df.sample(frac=1).reset_index(drop=True)  # Shuffle data

# Preprocessing
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=300)
y = np.array(df['target'])

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=300),
    SpatialDropout1D(0.2),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluate Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Predict Function
def predict_fake_news(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=300)
    prediction = model.predict(padded)[0][0]
    return "Fake News" if prediction > 0.5 else "Real News"





C:\Users\vardh\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 595s 1s/step - accuracy: 0.8957 - loss: 0.2232 - val_accuracy: 0.9910 - val_loss: 0.0372
Epoch 2/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 435s 773ms/step - accuracy: 0.9931 - loss: 0.0284 - val_accuracy: 0.9910 - val_loss: 0.0336
Epoch 3/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 303s 538ms/step - accuracy: 0.9969 - loss: 0.0129 - val_accuracy: 0.9923 - val_loss: 0.0316
Epoch 4/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 324s 577ms/step - accuracy: 0.9981 - loss: 0.0068 - val_accuracy: 0.9939 - val_loss: 0.0260
Epoch 5/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 350s 623ms/step - accuracy: 0.9963 - loss: 0.0118 - val_accuracy: 0.9927 - val_loss: 0.0308
281/281 ━━━━━━━━━━━━━━━━━━━━ 34s 118ms/step
Test Accuracy: 0.9927


In [10]:
# Example Prediction
print(predict_fake_news("The media has been talking all day about Trump and the Republican Party s scam of a tax bill; as well as the sheer obsequiousness of Trump s cabinet, and then members of Congress, after their tax scam was all but passed. But the media isn t quite saying what Trump wants. They ve been doing analysis and discussion all day long rather than praising it for the grand achievement Trump believes it is. The GOP has increasingly sounded exactly like Trump when it comes to media coverage, and coverage of the tax scam is no different. Coverage of Trump in general hasn t changed.Today, Lindsey Graham went after the media for portraying Trump as a  kook,  and unfit for office (they wouldn t be doing their job if they weren t telling the truth, though). Graham said: You know what concerns me about the American press is this endless, endless attempt to label the guy as some kind of kook; not fit to be president. Jake Tapper notes that he himself has never labeled Trump that way. But then he points out something rather odd about Graham s opinion. Take a look at the short video clip below:Lindsey Graham today: I m concerned by the media s attempt to label Trump as a kook or not fit to be President.Lindsey Graham in 2016:  I think he s a kook. I think he s crazy. I think he s unfit for office.  pic.twitter.com/hIxs3DciO8  Tomthunkit  (@TomthunkitsMind) December 17, 2017There it is, out of Graham s own mouth. He parroted himself. In 2016, he used the exact words to describe Trump that he said the media is using today. Freudian slip?Featured image via video screen capture"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Fake News
